### Recomendação musical em decorrência do mapeamento facial

### Input (vídeo/foto)
_______

### Dedução da emoção predominante
______

### Recomendação musical 

________

### Resposta do usuário

_______

### Cálculo do erro
________